# Collecting summoner IDs per Region, Division & Tier

In [ ]:
## Notes

Source: https://riot-api-libraries.readthedocs.io/en/latest/collectingdata.html

You can also use the League endpoints to get lists of ranked summoners. The positional league endpoints provide a paginated list of all summoners in a Tier + Division + Position (e.g. all ranked Diamond II Top laners). Alternatively, Canisback on the Discord currently hosts a list of league IDs that you can use to pull summoners from the leagues/{leagueId} endpoints. These lists are provided for free to the community for use, and may go down or stop being updated at any time.

https://developer.riotgames.com/apis#league-exp-v4/GET_getLeagueEntries


Helpful tutorial for this
- pulling specific data from API https://www.dataquest.io/blog/python-api-tutorial/ 


List of public API's for playing around another time
- https://github.com/public-apis/public-apis



Other (unused so far) tutorials:
- Writing & reading Json files https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/ 



In [ ]:
# to do

#WIP
1. Get summoner IDs & names by region, tier & division https://developer.riotgames.com/apis#league-exp-v4/GET_getLeagueEntries


# To do
2. Get account ID by summoner ID or name https://developer.riotgames.com/apis#summoner-v4/GET_getBySummonerName
3. Get matchlist by account ID https://developer.riotgames.com/apis#match-v4/GET_getMatchlist
    Can filter by the endTime and beginTime --> 	The end time to use for filtering matchlist specified as epoch milliseconds. If beginTime is specified, but not endTime, then endTime defaults to the the current unix timestamp in milliseconds (the maximum time range limitation is not observed in this specific case). If endTime is specified, but not beginTime, then beginTime defaults to the start of the account's match history returning a 400 due to the maximum time range limitation. If both are specified, then endTime should be greater than beginTime. The maximum time range allowed is one week, otherwise a 400 error code is returned.


In [ ]:
# Notes

json.dumps() — Takes in a Python object, and converts (dumps) it to a string.
json.loads() — Takes a JSON string, and converts (loads) it to a Python object.

jprint makes the output vertical instead of horizontal (better readable for dictionaries), but you need to make it first:
    # create a formatted string of the Python JSON object
    def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)


In [ ]:
# tutorial source: https://www.dataquest.io/blog/python-api-tutorial/ 

# import requests
# import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

parameters = {
    "lat": 40.71,
    "lon": -74}

response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)
jprint(response.json()) #prints the whole response
# jprint(response.json()['message']) #prints only the [] part of the response

pass_times = response.json()['response']
jprint(pass_times)

risetimes = []
for d in pass_times:
    time = d['risetime']
    risetimes.append(time)
    print (risetimes)

duradura = []
for b in pass_times:
    dur = b['duration']
    duradura.append(dur)
    print (duradura)

print (risetimes)
print (duradura)

In [ ]:
# tutorial source: https://towardsdatascience.com/json-and-apis-with-python-fba329ef6ef0

import requests
import pandas as pd

#response = requests.get('https://api.covid19api.com/summary').text
#response_info = json.loads(response)
#print ('responseinfo:', response_info)

response2 = requests.get('https://api.covid19api.com/summary').json()
#jprint(response2)

country_list = []
for country_info in response2['Countries']:
    country_list.append([country_info['Country'], country_info['TotalConfirmed']])
#jprint (country_list)

country_df = pd.DataFrame(data=country_list, columns=['Country', 'TotalConfirmed'])
# country_df.head(10)

In [ ]:
# this works
# importing challenger I 

import requests
import pandas as pd
import json

# create a formatted string of the Python JSON object
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)


## something here to modify the URL 
url_id = 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key=' + apikey

response = requests.get(url_id).json()
# jprint (response)

#grabbing summonerId, summonerName, wins, & losses and appending them to a new list summoner_list
summoner_list = []
for summoner_info in response:
    summoner_list.append([summoner_info['summonerId'], summoner_info['summonerName'], summoner_info['wins'], summoner_info['losses']])
    #jprint(summoner_list)

#creating a dataframe with the results
summoner_df = pd.DataFrame(data=summoner_list, columns=['summonerId', 'summonerName', 'wins', 'losses'])
#displaying the first 10 results of the dataframe
summoner_df.head(1000)


In [ ]:
# working in here
#  importing grandmaster I 

import requests
import pandas as pd
import json

# create a formatted string of the Python JSON object
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

## creating the URL to pull the data from - WIP
URL_ids = ('https://' + y + '.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/' +
                           tierList[z][0] + '/' + tierList[z][1] + '/?page=' + str(page) + '&api_key=' + apikey)

url_ids = ('https://' + region + '.api.riotgames.com/lol/league-exp/v4/entries/' + queue_type + 

final form: 'https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key=' + apikey

# set region
region = 'euw1'

# set queue type
queue_type = 'RANKED_SOLO_5x5'

# iterate through the tiers
tier_list = {'IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER'}

division_counter = 1
for tier_counter in tier_list:      # iterate through the tiers
        print (tier_counter)
        division_counter = 1
        while division_counter < 5: # iterate through the divisions. There are 4 divisions so until but excluding 5. 
            print (division_counter)
            division_counter = division_counter + 1



## iterate through the pages
####
pages = 

# pseudo code
for tier_list ...
    for division_list ... 
        for pages ... 
        if there is stuff in here, write it, 
        else if no data, stahp










# ///// from under here it works (direct copy from challenger I code)

response = requests.get(url_ids).json()
# jprint (response)

#grabbing summonerId, summonerName, wins, & losses and appending them to a new list summoner_list
summoner_list = []
for summoner_info in response:
    summoner_list.append([summoner_info['summonerId'], summoner_info['summonerName'], summoner_info['wins'], summoner_info['losses']])
    #jprint(summoner_list)

#creating a dataframe with the results
summoner_df = pd.DataFrame(data=summoner_list, columns=['summonerId', 'summonerName', 'wins', 'losses'])
#displaying the first 10 results of the dataframe
summoner_df.head(1000)


In [70]:
url_ids = ('https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key=' + apikey)


# ///// working under here 

response = requests.get(url_ids).json()
jprint (response)

SILVER
1
2
3
4
DIAMOND
1
2
3
4
GRANDMASTER
1
2
3
4
IRON
1
2
3
4
MASTER
1
2
3
4
CHALLENGER
1
2
3
4
GOLD
1
2
3
4
BRONZE
1
2
3
4
PLATINUM
1
2
3
4


In [ ]:
# iterate through the tiers
tier_list = {'IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER'}

division_counter = 1
for tier_counter in tier_list:      # iterate through the tiers
        print (tier_counter)
        division_counter = 1
        while division_counter < 5: # iterate through the divisions. There are 4 divisions so until but excluding 5. 
            print (division_counter)
            division_counter = division_counter + 1
